In [1]:
import numpy as np
import os
import torch
from torch import nn, optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

Using device cuda:0
GeForce GTX 1660 SUPER
P106-100


In [2]:
from torch.utils.data import Dataset, DataLoader
from data.MinicityDataset import TrafficDataset

dbpath = '/home/rong/disk/database/minicity.db'
bs = 12
train_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = True),
                         batch_size = bs, shuffle=True, drop_last = True)
test_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = False),
                          batch_size = bs, shuffle=True, drop_last = True)

# batch = next(iter(train_loader))
# batch['data']

In [3]:
from models.minicity import convVAE

X_dim = 4
traj_size = 25 * 4
z_dim = 50
cnn_out_size = 300
cond_out_size = 300

model = convVAE(sample_size = X_dim, 
                  traj_size = traj_size,
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)

checkpoint = torch.load('checkpoints/minicity_attention_7.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
print(model)

convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap_pool2): AdaptiveAvg

In [6]:
from utils.Minicity import plotData, plotOrientSpeed, plotAlpha
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
viz_idx = 8753 #弯道
# viz_idx = 143300 #弯道
# viz_idx = 5107 #直行
# viz_idx = 2984 #转盘
# viz_idx = 101538 #转盘
# viz_idx = 153309 #阻塞
# viz_idx = 109779 #变道

print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ[:, 200:600, 200:600]        
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
occ = adap_pool(occ)
occ = occ.to(device)
traj = torch.from_numpy(batch["traj"]).to(device)
print(traj.shape)
data = torch.from_numpy(batch["data"]).to(device)
egoid = batch["egoid"]
print(traj.shape)
time_stamp = batch["timeprob"]
print(time_stamp)

with torch.no_grad():
    model.eval()
    y_viz = torch.randn(1,4).to(device)
    for i in range(0, 10):
        num_viz = 12
        y_viz_p, alpha = model.inference(startgoal.expand(num_viz, X_dim * 2).to(device), traj.expand(num_viz, 25, 4),
                                occ.expand(num_viz, 1, -1, -1, -1).to(device), num_viz)
        torch.cuda.empty_cache()
        y_viz = torch.cat((y_viz_p, y_viz), dim = 0)

y_viz=y_viz.cpu().detach().numpy()*50
occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy() * 50
data=data.cpu().detach().numpy() * 50
alpha=alpha.cpu().detach().numpy()
torch.cuda.empty_cache()
# from utils.NarrowPassage import plotCondition, plotSample, plotSpeed, plotSampleAttention

%matplotlib
# from utils.HighWay import plotData, plotOrientSpeed, plotAlpha
    
y_viz=y_viz[:-1]
plotData(occ, startgoal, y_viz)
plotOrientSpeed(startgoal, y_viz)
# plotAlpha(alpha)

8753
torch.Size([25, 4])
torch.Size([25, 4])
1276.0
Using matplotlib backend: Qt5Agg
[  0.         0.        -1.4532     9.86      16.669998 -20.279999
  -0.3976    11.98    ]
155.56666056315106 167.5999959309896


In [8]:
plotAlpha(alpha)

In [17]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
#  
print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ[:, 200:600, 200:600]        
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
occ = adap_pool(occ)
# adap_pool = nn.AdaptiveAvgPool3d((25,100, 600))
# occ = adap_pool(occ)
occ = occ.to(device)
data = torch.from_numpy(batch["data"]).to(device)

occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy() * 50
data=data.cpu().detach().numpy() * 50
torch.cuda.empty_cache()

plotData(occ, startgoal, data)

68844
[ 0.         0.        -0.         1.72       4.1299973  0.
 -0.         2.32     ]
113.76665751139323 100.0


# commonroad collision

In [1]:
import os
import matplotlib.pyplot as plt

from commonroad.common.file_reader import CommonRoadFileReader
from commonroad_cc.visualization.draw_dispatch import draw_object

file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/minicity.cr.xml'
# file_path = "scenarios/commonroad_data/ZAM_Tutorial-1_2_T-1.xml"
# file_path = "/home/rong/commonroad/commonroad-search-master/GSMP/tools/commonroad-road-boundary/scenarios/GER_Ffb_2.xml"
scenario, _ = CommonRoadFileReader(file_path).open()

In [5]:
# plot the scenario
plt.figure(figsize=(25, 10))
draw_object(scenario)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()

In [9]:
from commonroad.scenario.scenario import Scenario
from commonroad.scenario.trajectory import State as StateTupleFactory
from commonroad.scenario.obstacle import StaticObstacle, ObstacleType, Obstacle
import numpy as np
from commonroad.geometry.shape import Polygon, ShapeGroup, Circle
from commonroad_cc.collision_detection.pycrcc_collision_dispatch import create_collision_checker, create_collision_object
import pycrcc

carcc = create_collision_checker(scenario)

from scenarios.commonroad_road_boundary.construction import construct

%matplotlib

build = ['section_triangles']
boundary = construct(scenario, build, ['section_triangles'], ['plot'])

road_boundary_shape_list = list()
initial_state = None
for r in boundary['section_triangles'].unpack():
    initial_state = StateTupleFactory(position=np.array([0, 0]), orientation=0.0, time_step=0)
    p = Polygon(np.array(r.vertices()))
    road_boundary_shape_list.append(p)
road_bound = StaticObstacle(obstacle_id=scenario.generate_object_id(), obstacle_type=ObstacleType.ROAD_BOUNDARY,
                            obstacle_shape=ShapeGroup(road_boundary_shape_list), initial_state=initial_state)

roadcc = pycrcc.CollisionChecker()
roadcc.add_collision_object(create_collision_object(road_bound))

In [17]:
import numpy as np
from commonroad.scenario.trajectory import State, Trajectory
from commonroad.prediction.prediction import TrajectoryPrediction
from commonroad.geometry.shape import Rectangle
from commonroad.scenario.obstacle import StaticObstacle
from commonroad_cc.collision_detection.pycrcc_collision_dispatch import create_collision_object

# # create a trajectory for the ego vehicle starting at time step 0
time_start = 0
position = np.array([[50, -6]])
orientation = [0]
state_list = list()
for k in range(0, len(position)):
    state_list.append(State(**{'position': position[k], 'orientation':orientation[k]}))
trajectory = Trajectory(time_start, state_list)
# create the shape of the ego vehicle
shape = Rectangle(length=3.9, width=1.9)
# # create a TrajectoryPrediction object consisting of the trajectory and the shape of the ego vehicle
traj_pred = TrajectoryPrediction(trajectory=trajectory, shape=shape)
# create a collision object using the trajectory prediction of the ego vehicle
co = create_collision_object(traj_pred)

# test the trajectory of the ego vehicle for collisions
print('Collision between the trajectory of the ego vehicle and objects in the environment: ', carcc.collide(co))
print('Collision between the trajectory of the ego vehicle and road: ', roadcc.collide(co))

Collision between the trajectory of the ego vehicle and objects in the environment:  False
Collision between the trajectory of the ego vehicle and road:  True
